# Projet informatique - Mazars
## BELAMRI Pierre
## PLANCHE Louis
## DEJOIE Antoine

1) Récupération et parsing des quotations d’options
Un jeu de données au format .csv sera fourni aux étudiants. Il contiendra des données de marché tel qu’observable sur le site https://www.barchart.com/options . Les étudiants développeront un parser, qui filtrera le fichier d’input pour en conserver seulement les données utiles. 


Le parser se trouve dans le fichier $ parser.py  $ et s'utilise par le terminal de la façon suivante :

- Ouvrir le terminal
- Se placer dans le répertoire contenant parser.py ainsi que le csv à trier.

*cd ~path/projet_mazars/*

- Appeler le programme en mettant en argument le nom du csv à trier, le symbole sur lequel on travaille, l'option (Call, Put ou Both par défaut pour les deux)

*python parser.py stocks-options.csv AAPL Call*

- Vous serez ensuite invité à saisir une valeur pour le DTE

*11*

- Pour cet exemple, le programme créer dans le même répertoire un fichier "stocks-options_extracterd_AAPL.csv" contenant seulement les lignes concernant Apple pour des options d'achats qui expirent dans 11 jours.

Le programme fonctionne de la façon suivante: Les lignes du csv vont être lues une par une (*for row in spamreader*) jusqu'a ce que le symbol, le DTE et le choix de l'option correspondent avec notre choix (*if*) dans ce cas on balaie toute la ligne avec une boucle *for* et on créé un dictionnaire contenant toute cette ligne, on écrit ensuite ce dictionnaire dans un nouveau csv ( *writer.writerow(d)*)

In [ ]:
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')

for row in spamreader:
    A = list((",".join(row)).split(","))
    if A[0] == symbol  and A[5] == str(DTE) and (A[2] == choix) :
        d=dict()
        for j in range (len(A)):
            d[B[j]] = A[j] 
        writer.writerow(d)

## BELAMRI Pierre
## PLANCHE Louis
## DEJOIE Antoine

In [ ]:
# test
## test